### Import packages

In [1]:
import pandas as pd
import pyarrow.csv as pv
import pyarrow.parquet as pq

### Convert data to parquet

In [6]:
city_neighborhood = pv.read_csv('./data/City_of_Edmonton_-_Neighbourhoods_20240201.csv')
city_property_info = pv.read_csv('./data/Property_Information__Current_Calendar_Year__20240201.csv')
city_property_assessment = pv.read_csv('./data/Property_Assessment_Data__Current_Calendar_Year__20240201.csv')

pq.write_table(city_neighborhood,'./data/city_neighborhood.parquet')
pq.write_table(city_property_info,'./data/city_property_info.parquet')
pq.write_table(city_property_assessment,'./data/city_property_assessment.parquet')